# 1. Install and Import Dependencies

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import glob
import pandas as pd
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'



# 2. Data Processing

## 2.1 Youtube data processing

In [14]:
# List of folder names (car brands)
folders = ["Alfa Romeo", "Audi Q5", "BMW X3", "Grecale", "mercedes GLC", "Porsche Macan"]

list_of_dfs = []

# Iterate over each folder and get CSV files
for folder in folders:
    path = f"/Users/apple/Desktop/test/{folder}/*.csv"
    all_files = glob.glob(path)
    
    # Read each file in the current folder
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        
        # Add the "Car brand" column with the folder's name
        df["Car brand"] = folder
        
        # Append the dataframe to the list_of_dfs
        list_of_dfs.append(df)

# Concatenate all dataframes in the list into one dataframe
combined_df = pd.concat(list_of_dfs, axis=0, ignore_index=True)


## 2.2 Combine all sources result 

In [15]:
df_youtube = combined_df
df_youtube.rename(columns={'id': 'Name', 'Car brand': 'Model', 'date': 'Date'}, inplace=True)

# convert "Alfa Romeo" to "Alfa Romeo Stelvio"
df_youtube['Model'] = df_youtube['Model'].replace('Alfa Romeo', 'Alfa Romeo Stelvio')
# convert "Grecale" to "Maserati Grecale"
df_youtube['Model'] = df_youtube['Model'].replace('Grecale', 'Maserati Grecale')

df_youtube['Source'] = 'Youtube'  # 新增Source欄位，標示來源

# 讀取CombinedPistonheads.csv文件，並新增Source欄位
df_pistonheads = pd.read_csv('/Users/apple/Desktop/CombinedPistonheads.csv')
df_pistonheads['Source'] = 'Pistonheads'

# 讀取CombinedForums (3).csv文件，並新增Source欄位
df_forums = pd.read_csv('/Users/apple/Desktop/CombinedForums (3).csv')
df_forums['Source'] = 'Forums'

# 合併數據集
combined_df = pd.concat([df_youtube, df_pistonheads, df_forums], ignore_index=True)

# 顯示合併後的數據集
print(combined_df)


       Unnamed: 0                                            Comment  \
0             0.0  What do you think of the Alfa Romeo Stelvio Qu...   
1             1.0  I'm not sure I have ever commented on a video,...   
2             2.0  I’ve always done the cup holder test in all my...   
3             3.0  I absolutely love the design of the Stelvio an...   
4             4.0  I believe the reason behind the column mounted...   
...           ...                                                ...   
60856         NaN  I suppose we should be grateful that at least ...   
60857         NaN  Our first macan was a base on 21s with no Pasm...   
60858         NaN  For gen3, on the uk configurator PASM is not c...   
60859         NaN  Well that is another reason for avoiding the 2...   
60860         NaN  We don’t do too bad. At current exchange rates...   

                           Name                  Date    Pos    Neg    Neu  \
0                 \n @ogfeez\n   4 years ago (edited)  0.

## 2-3 Generate Sentiment Scores for all sources

??

In [16]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Function to get sentiment score
def get_sentiment_score(text):
    
    if not isinstance(text, str):
        return -1  # or any placeholder value for invalid inputs
    
    tokens = tokenizer.encode(text, add_special_tokens=True)  # Add special tokens for BERT (e.g., [CLS], [SEP])
    
    # Check token length
    if len(tokens) > 512:
        return -1  # or any other placeholder value
    
    # Get sentiment score
    with torch.no_grad():  # Added to ensure gradients aren't computed
        result = model(torch.tensor([tokens]))  # Converted tokens to tensor and added extra dimension
    return int(torch.argmax(result.logits)) + 1


In [20]:

# Read the CSV
df = combined_df
df['sentiment_score'] = ''
# Fill NaN with empty string
df['Comment'].fillna("", inplace=True)
df['Comment'] = df['Comment'].astype(str)

# # Assuming 'comment' is the column name for the comments
# df['sentiment_score'] = df['Comment'].apply(get_sentiment_score)


for idx, i in enumerate(df['Comment']):
    if idx % 1000 == 0:
        print(f"Already generate {idx} times.")
    df['sentiment_score'].iloc[idx] = get_sentiment_score(df['Comment'].iloc[idx])
    

Already generate 0 times.
Already generate 1000 times.
Already generate 2000 times.
Already generate 3000 times.
Already generate 4000 times.
Already generate 5000 times.
Already generate 6000 times.
Already generate 7000 times.
Already generate 8000 times.
Already generate 9000 times.
Already generate 10000 times.
Already generate 11000 times.
Already generate 12000 times.
Already generate 13000 times.
Already generate 14000 times.
Already generate 15000 times.
Already generate 16000 times.
Already generate 17000 times.
Already generate 18000 times.
Already generate 19000 times.
Already generate 20000 times.
Already generate 21000 times.
Already generate 22000 times.
Already generate 23000 times.
Already generate 24000 times.
Already generate 25000 times.
Already generate 26000 times.
Already generate 27000 times.
Already generate 28000 times.
Already generate 29000 times.
Already generate 30000 times.
Already generate 31000 times.
Already generate 32000 times.
Already generate 33000 

## 2-4 Sentiment Scores Processing & Saving

In [21]:
# 1. 计算sentiment_score为-1的行数
num_rows_with_minus_one = len(df[df['sentiment_score'] == -1])

# 2. 去除这些行
df = df[df['sentiment_score'] != -1]

# 3. 计算被删除行的百分比
total_rows = len(df)
percentage_removed = (num_rows_with_minus_one / (num_rows_with_minus_one + total_rows)) * 100

# 顯示被刪除行的百分比
print(f"Percentage of removed rows: {percentage_removed:.2f}%")

# 4. Check if the column exists and drop it
if 'column' in df.columns:
    df = df.drop('column', axis=1)

## df -> only contain YT with scores
print(df)

# Save the dataframe back to CSV  
df.to_csv('/Users/apple/Desktop/combined_final_3.csv', index=False)

Percentage of removed rows: 0.29%
       Unnamed: 0                                            Comment  \
1             1.0  I'm not sure I have ever commented on a video,...   
2             2.0  I’ve always done the cup holder test in all my...   
3             3.0  I absolutely love the design of the Stelvio an...   
4             4.0  I believe the reason behind the column mounted...   
5             5.0  Finally! As a previous 4C Spider and now Giuli...   
...           ...                                                ...   
60856         NaN  I suppose we should be grateful that at least ...   
60857         NaN  Our first macan was a base on 21s with no Pasm...   
60858         NaN  For gen3, on the uk configurator PASM is not c...   
60859         NaN  Well that is another reason for avoiding the 2...   
60860         NaN  We don’t do too bad. At current exchange rates...   

                           Name              Date    Pos    Neg    Neu  \
1      \n @charlesconnor173